# 기사 크롤링하기

In [7]:
import pandas as pd
# 상한가 CSV 파일 읽기
sanghanga_df = pd.read_csv('./datas/real_final_sanghanga_df.csv')
sanghanga_df.head()


,key,날짜,구분,종목코드,종목명,현재가,전일대비,등락률,거래량,거래대금,시가,고가,저가,10일변동가격,10일변동비율,익일시가,10일고가,라벨
0,0,20180102,신,85660,차바이오텍,29250.0,6750.0,30.00,13003908,356270,23300.0,29250.0,23150.0,11850.0,0.39,30150,42000,1
1,1,20180102,증,900100,애머릿지,3185.0,735.0,30.00,49558279,144720,2600.0,3185.0,2545.0,1290.0,0.37,3500,4790,1
2,2,20180102,NaN,41190,우리기술투자,4360.0,1005.0,29.96,45431164,180938,3560.0,4360.0,3500.0,5380.0,1.14,4720,10100,1
3,3,20180103,신,263810,상신전자,27950.0,6450.0,30.00,1404491,36493,22000.0,27950.0,22000.0,3950.0,0.14,27500,31450,0
4,4,20180103,증,56730,CNT85,3555.0,820.0,29.98,4566984,14295,2860.0,3555.0,2680.0,540.0,0.15,3650,4190,0


In [3]:
# 크롬을 통해 크롤링 하기 위해 웹 드라이버 import
from selenium import webdriver

# 구글 드라이버 자동설치
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# 요소 찾을 때 사용하는 By 모듈
from selenium.webdriver.common.by import By

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys

# 페이지 렌더링 될 때까지 기다리게 하려고 쓰는 time 함수
import time
import random
t1 = random.uniform(1,3) # 1~3초 사이
t2 = random.uniform(3,5) # 2~5초 사이
t3 = random.uniform(5,8) # 5~8초 사이

# 브라우저 컨트롤 통신 관련하여 설정하려고 쓰는 모듈
import ssl

# 데이터 프레임 만들려고
import pandas as pd

# 반복 알아보기
from tqdm.notebook import tqdm

# 브라우저 컨트롤할때 반영할 옵션들
# options = Options()
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_experimental_option("detach", True)
# 봇 탐지 방어 코드..
options.add_argument("--disable-blink-features=AutomationControlled")
# 크롤링 창 사이즈 고정
options.add_argument("window-size=1920x1080")
# 원래 셀레니움은 창이 뜨는 상태로 크롤링됨, 하지만 headless 옵션을 추가하면 창이 안떠도 암묵적으로 크롤링이 된다.
# options.add_argument("headless")

# 창 크기 최대화
# driver.maximize_window()

# ssl 인증서 검증을 비활성화, 가끔 셀레니움으로 켯을 때 사이트 자체에서 ssl 인증을 허가 하지 않는 접속을 막아버린다.
ssl._create_default_https_context = ssl._create_unverified_context

# 브라우저를 컨트롤 할 드라이버 객체를 선언, 원래는 크롬드라이버가 설치되어야 하지만, 아래 문구로 엄청 쉽게 사용할 수 있다.
driver = webdriver.Chrome(options=options)

# 페이지 3초동안 로드될때까지 기다림, 안켜지면 오류 발생시킴 암묵적 wait
driver.implicitly_wait(10)

""" # 주소창 접속
driver.get("https://www.daum.net/")
time.sleep(t1)

# 종목 검색
driver.find_element(By.XPATH, '//*[@id="q"]').send_keys(keyword)
time.sleep(t1)

# 검색 버튼 클릭
driver.find_element(By.XPATH, '//*[@id="daumSearch"]/fieldset/div/div/button[3]').click()
time.sleep(t1)

# 뉴스탭 들어가기
driver.find_element(By.XPATH, '//*[@id="daumGnb"]/div[1]/ul/li[2]/a').click()
time.sleep(t1)
"""

# 상한가 CSV 파일 읽기
sanghanga_df = pd.read_csv('./datas/real_final_sanghanga_df.csv')

# 시작위치 재설정
sanghanga_df = sanghanga_df[10:]

# 종목 셋팅
# keyword = input('검색할 종목명을 입력하세요: ')
# goodday = input('상한가 도달한 날짜를 입력하세요: YYYYMMNN')
keys = sanghanga_df['key']
names = sanghanga_df['종목명']
sanghanga_df['종목코드'] = sanghanga_df['종목코드'].apply(lambda x: '{:06d}'.format(x)) # 자릿수 유지
keywords = sanghanga_df['종목코드']
gooddays = sanghanga_df['날짜']
labels = sanghanga_df['라벨'] # 상한가 날 이후 10일 동안 최대 20% 상승 했는지 아닌지 [0: 추가 상승 없음, 1: 추가 상승 있음]

for key, name, keyword, goodday, realUp in tqdm(zip(keys, names, keywords, gooddays, labels)): # 데이터셋에서 종목명 기준으로 반복.
    # df에 담을 정보
    nums = [] # 넘버
    days = [] # 날짜
    stocks = [] # 종목 명
    codes = [] # 종목 코드
    titles = [] # 뉴스 제목
    hrefs = [] # 뉴스 주소
    contexts = [] # 뉴스 본문
    realUps = [] # 종목 진짜 올랐는지.
    
    keyword = str(keyword)
    goodday = str(goodday)
    goodday2 = f"{goodday[:4]}.{goodday[4:6]}.{goodday[6:8]}"
    # 주식 종목 조건에 맞게 검색 [종목코드, 상한가 날짜]
    url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=news&ssc=tab.news.all&query='+'%s'%keyword+'+-로봇기자+-뉴스봇+-Biz봇&oquery='+'%s'%keyword+'+-로봇기자+-뉴스봇&tqi=ip8rEsqo1e8ssgkadkKssssstXV-383846&nso=so%3Ar%2Cp%3Afrom'+'%s'%goodday+'to'+'%s'%goodday+'&de='+'%s'%goodday2+'&ds='+'%s'%goodday2+'&mynews=1&office_category=0&office_section_code=0&office_type=0&pd=3&photo=0&service_area=1&sort=0'
    
    driver.get(url)
    time.sleep(t3)

    # 스크롤 내리는 기능 입니다.
    actions = driver.find_element(By.CSS_SELECTOR, 'body')
    for i in range(2): #스크롤 내릴 횟수 설정
        actions.send_keys(Keys.END)
        time.sleep(1)
        
    # 종목 뉴스 검색했을 때의 URL을 나중에 되돌아올 수 있도록 따로 보관해 두기.
    # bowl_url = driver.current_url

    #뉴스 title, href 추출
    elements = driver.find_elements(By.CSS_SELECTOR, "#main_pack > section.sc_new.sp_nnews._fe_news_collection._prs_nws > div.api_subject_bx > div.group_news > ul > li")
    if len(elements) != 0 : # 뉴스가 없는 경우도 있음.
        for element in elements:
            try:
                href_temp = element.find_element(By.CSS_SELECTOR, "div.info_group > a:nth-child(3)")
                hrefs.append(href_temp.get_attribute("href")) #뉴스링크
                realUps.append(realUp) # 라벨
                stocks.append(name) #주식명
                nums.append(key)
                codes.append(keyword)
                days.append(goodday)
                element = element.find_element(By.CSS_SELECTOR, "div.news_contents")
                element = element.find_element(By.CSS_SELECTOR, "a.news_tit")
                titles.append(element.text) #뉴스제목
            except : pass
            
        # 뉴스 링크에 접속해서 본문 따기
        for i in range(0,len(hrefs)):
            driver.get(hrefs[i])
            time.sleep(t2)
            try : 
                context = driver.find_element(By.XPATH, '//*[@id="dic_area"]').text
            except : # 본문 구조가 다른 경우가 있음
                context = '기사가 없쪄용'
            context = context.replace("\n", " ")
            contexts.append(context)
            time.sleep(t1)

        # 데이터프레임 생성
        df = pd.DataFrame({
            'key': nums,
            'realUp': realUps,
            'stock': stocks,
            'code' : codes,
            'day' : days,
            'title': titles,
            'href': hrefs,
            'context': contexts
        })

        # CSV 파일로 저장
        df.to_csv('./datas/네이버_뉴스기사/naver_news_origin.csv', mode='a' ,header=False, index=False)
        
        time.sleep(t1)

print('뉴스 데이터 수집이 완료되었습니다.')

0it [00:00, ?it/s]

뉴스 데이터 수집이 완료되었습니다.


# 중복된 기사 제거

In [1]:
import pandas as pd
# 수집한 CSV 파일 읽기
naver_news_origin_df = pd.read_csv('./datas/네이버_뉴스기사/naver_news_origin.csv',dtype={'code': str})
naver_news_origin_df # 10825 row

,key,realUp,stock,code,day,title,href,context
0,0,1,차바이오텍,085660,20180102,차바이오그룹 회장으로 송재훈 전 삼성서울병원장 선임,https://n.news.naver.com/mnews/article/011/000...,[서울경제] 차바이오그룹은 송재훈 전 삼성서울병원장을 신임 회장으로 선임했다고 2일...
1,0,1,차바이오텍,085660,20180102,"차바이오텍, 美 보조금 집행으로 수익성 개선 전망, 52주 신고가",https://n.news.naver.com/mnews/article/366/000...,바이오기업 차바이오텍의 실적 기대감이 고조되고 있다. 2017년 4분기 실적부터 미...
2,0,1,차바이오텍,085660,20180102,"[마켓뷰] 약달러에 경기민감주 강세…""업종 순환매 이어질 것""",https://n.news.naver.com/mnews/article/366/000...,"새해 첫 증시는 상승장이었다. 조선, 건설, 철강 등 경기 민감주가 강세를 나타낸 ..."
3,0,1,차바이오텍,085660,20180102,"[마켓뷰] 국내 증시, 연초 기대감에 상승…코스닥 800선 회복",https://n.news.naver.com/mnews/article/366/000...,국내 증시가 연초 기대감에 상승세를 나타내고 있다. 주로 외국인과 기관보다는 개인 ...
4,2,1,우리기술투자,041190,20180102,"[특징주]우리기술투자, 두나무 투자 '잭팟' 강세",https://n.news.naver.com/mnews/article/018/000...,[이데일리 박형수 기자] 우리기술투자(041190)가 급등하고 있다. 가상화폐 거래...
...,...,...,...,...,...,...,...,...
10820,4528,0,국순당,043650,20240611,[코스닥 마감]나스닥 훈풍에 870선 목전…알테오젠 시총 2위 탈환,https://n.news.naver.com/mnews/article/018/000...,"전거래일 댑, 0.42%↑, 868.36 장마감 외인·기관 매수에 16일 만에 장중..."
10821,4530,0,코오롱글로벌,003070,20240612,[특징주] 포항 영일만항 확장 소식에 코오롱글로벌 上,https://n.news.naver.com/mnews/article/018/000...,"코오롱글로벌, 코오롱글로벌우 둘다 상한가 [이데일리 김소연 기자] 코오롱글로벌(00..."
10822,4530,0,코오롱글로벌,003070,20240612,"[ET라씨로] 코오롱글로벌, 포항 영일만항 확장 소식에 상한가로",https://n.news.naver.com/mnews/article/030/000...,ET라씨로는 인공지능(AI) 기반으로 선별·분석한 주식 정보와 종목 매매 신호를 제...
10823,4531,0,에스앤디,260970,20240613,"에스앤디, 불닭 열풍에 동반성장…동종업계 대비 저평가 -KB",https://n.news.naver.com/mnews/article/018/000...,[이데일리 김보겸 기자] KB증권은 에스앤디(260970)에 대해 불닭볶음면 열풍으...


In [2]:
naver_news_origin_df = naver_news_origin_df.drop_duplicates(subset='context', keep='first')
naver_news_origin_df # 7889 row

,key,realUp,stock,code,day,title,href,context
0,0,1,차바이오텍,085660,20180102,차바이오그룹 회장으로 송재훈 전 삼성서울병원장 선임,https://n.news.naver.com/mnews/article/011/000...,[서울경제] 차바이오그룹은 송재훈 전 삼성서울병원장을 신임 회장으로 선임했다고 2일...
1,0,1,차바이오텍,085660,20180102,"차바이오텍, 美 보조금 집행으로 수익성 개선 전망, 52주 신고가",https://n.news.naver.com/mnews/article/366/000...,바이오기업 차바이오텍의 실적 기대감이 고조되고 있다. 2017년 4분기 실적부터 미...
2,0,1,차바이오텍,085660,20180102,"[마켓뷰] 약달러에 경기민감주 강세…""업종 순환매 이어질 것""",https://n.news.naver.com/mnews/article/366/000...,"새해 첫 증시는 상승장이었다. 조선, 건설, 철강 등 경기 민감주가 강세를 나타낸 ..."
3,0,1,차바이오텍,085660,20180102,"[마켓뷰] 국내 증시, 연초 기대감에 상승…코스닥 800선 회복",https://n.news.naver.com/mnews/article/366/000...,국내 증시가 연초 기대감에 상승세를 나타내고 있다. 주로 외국인과 기관보다는 개인 ...
4,2,1,우리기술투자,041190,20180102,"[특징주]우리기술투자, 두나무 투자 '잭팟' 강세",https://n.news.naver.com/mnews/article/018/000...,[이데일리 박형수 기자] 우리기술투자(041190)가 급등하고 있다. 가상화폐 거래...
...,...,...,...,...,...,...,...,...
10820,4528,0,국순당,043650,20240611,[코스닥 마감]나스닥 훈풍에 870선 목전…알테오젠 시총 2위 탈환,https://n.news.naver.com/mnews/article/018/000...,"전거래일 댑, 0.42%↑, 868.36 장마감 외인·기관 매수에 16일 만에 장중..."
10821,4530,0,코오롱글로벌,003070,20240612,[특징주] 포항 영일만항 확장 소식에 코오롱글로벌 上,https://n.news.naver.com/mnews/article/018/000...,"코오롱글로벌, 코오롱글로벌우 둘다 상한가 [이데일리 김소연 기자] 코오롱글로벌(00..."
10822,4530,0,코오롱글로벌,003070,20240612,"[ET라씨로] 코오롱글로벌, 포항 영일만항 확장 소식에 상한가로",https://n.news.naver.com/mnews/article/030/000...,ET라씨로는 인공지능(AI) 기반으로 선별·분석한 주식 정보와 종목 매매 신호를 제...
10823,4531,0,에스앤디,260970,20240613,"에스앤디, 불닭 열풍에 동반성장…동종업계 대비 저평가 -KB",https://n.news.naver.com/mnews/article/018/000...,[이데일리 김보겸 기자] KB증권은 에스앤디(260970)에 대해 불닭볶음면 열풍으...


In [3]:
# CSV 파일로 저장
naver_news_origin_df.to_csv('./datas/네이버_뉴스기사/naver_news_origin_duplicates.csv', index=False)

In [5]:
naver_news_origin_duplicates = pd.read_csv('./datas/네이버_뉴스기사/naver_news_origin_duplicates.csv',dtype={'code': str})
naver_news_origin_duplicates

,key,realUp,stock,code,day,title,href,context
0,0,1,차바이오텍,085660,20180102,차바이오그룹 회장으로 송재훈 전 삼성서울병원장 선임,https://n.news.naver.com/mnews/article/011/000...,[서울경제] 차바이오그룹은 송재훈 전 삼성서울병원장을 신임 회장으로 선임했다고 2일...
1,0,1,차바이오텍,085660,20180102,"차바이오텍, 美 보조금 집행으로 수익성 개선 전망, 52주 신고가",https://n.news.naver.com/mnews/article/366/000...,바이오기업 차바이오텍의 실적 기대감이 고조되고 있다. 2017년 4분기 실적부터 미...
2,0,1,차바이오텍,085660,20180102,"[마켓뷰] 약달러에 경기민감주 강세…""업종 순환매 이어질 것""",https://n.news.naver.com/mnews/article/366/000...,"새해 첫 증시는 상승장이었다. 조선, 건설, 철강 등 경기 민감주가 강세를 나타낸 ..."
3,0,1,차바이오텍,085660,20180102,"[마켓뷰] 국내 증시, 연초 기대감에 상승…코스닥 800선 회복",https://n.news.naver.com/mnews/article/366/000...,국내 증시가 연초 기대감에 상승세를 나타내고 있다. 주로 외국인과 기관보다는 개인 ...
4,2,1,우리기술투자,041190,20180102,"[특징주]우리기술투자, 두나무 투자 '잭팟' 강세",https://n.news.naver.com/mnews/article/018/000...,[이데일리 박형수 기자] 우리기술투자(041190)가 급등하고 있다. 가상화폐 거래...
...,...,...,...,...,...,...,...,...
7884,4528,0,국순당,043650,20240611,[코스닥 마감]나스닥 훈풍에 870선 목전…알테오젠 시총 2위 탈환,https://n.news.naver.com/mnews/article/018/000...,"전거래일 댑, 0.42%↑, 868.36 장마감 외인·기관 매수에 16일 만에 장중..."
7885,4530,0,코오롱글로벌,003070,20240612,[특징주] 포항 영일만항 확장 소식에 코오롱글로벌 上,https://n.news.naver.com/mnews/article/018/000...,"코오롱글로벌, 코오롱글로벌우 둘다 상한가 [이데일리 김소연 기자] 코오롱글로벌(00..."
7886,4530,0,코오롱글로벌,003070,20240612,"[ET라씨로] 코오롱글로벌, 포항 영일만항 확장 소식에 상한가로",https://n.news.naver.com/mnews/article/030/000...,ET라씨로는 인공지능(AI) 기반으로 선별·분석한 주식 정보와 종목 매매 신호를 제...
7887,4531,0,에스앤디,260970,20240613,"에스앤디, 불닭 열풍에 동반성장…동종업계 대비 저평가 -KB",https://n.news.naver.com/mnews/article/018/000...,[이데일리 김보겸 기자] KB증권은 에스앤디(260970)에 대해 불닭볶음면 열풍으...
